In [31]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset
import time
import numpy as np

In [32]:
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
    ])
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
    ])
train_set = datasets.ImageFolder(root='data/001/train',transform=train_transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=4, shuffle=True, num_workers=8)

test_set = datasets.ImageFolder(root='data/001/test',transform=test_transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=4, shuffle=True, num_workers=8)

In [33]:
def train(model, device, criterion, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        outputs = model(data)
        loss = criterion(outputs, target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {:2d} [{:5d}/{} ({:2.1f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [34]:
def test(model, device, criterion, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            
            test_loss += criterion(output, target).item() 
            pred = output.argmax(dim=1)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [35]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available(): print(torch.cuda.get_device_name(0))

Tesla K80


In [36]:
model = models.resnet50(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
    
n = len(np.unique(train_set.targets))
classifier = nn.Sequential(nn.Linear(model.fc.in_features,128),
                          nn.ReLU(),
                          nn.Dropout(0.2),
                          nn.Linear(128,64),
                          nn.ReLU(),
                          nn.Dropout(0.2),
                          nn.Linear(64,n))
model.fc = classifier
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

for epoch in range(20):
    train(model, device, criterion, train_loader, optimizer, epoch)
    test(model, device, criterion, test_loader)

Train Epoch:  0 [    0/4654 (0.0%)]	Loss: 2.334228
Train Epoch:  0 [  400/4654 (8.6%)]	Loss: 2.129941
Train Epoch:  0 [  800/4654 (17.2%)]	Loss: 2.234255
Train Epoch:  0 [ 1200/4654 (25.8%)]	Loss: 1.423135
Train Epoch:  0 [ 1600/4654 (34.4%)]	Loss: 2.024154
Train Epoch:  0 [ 2000/4654 (43.0%)]	Loss: 2.172873
Train Epoch:  0 [ 2400/4654 (51.5%)]	Loss: 1.488317
Train Epoch:  0 [ 2800/4654 (60.1%)]	Loss: 1.632560
Train Epoch:  0 [ 3200/4654 (68.7%)]	Loss: 1.340322
Train Epoch:  0 [ 3600/4654 (77.3%)]	Loss: 1.500132
Train Epoch:  0 [ 4000/4654 (85.9%)]	Loss: 1.172336
Train Epoch:  0 [ 4400/4654 (94.5%)]	Loss: 1.467947

Test set: Average loss: 0.7333, Accuracy: 1129/1552 (73%)

Train Epoch:  1 [    0/4654 (0.0%)]	Loss: 0.765608
Train Epoch:  1 [  400/4654 (8.6%)]	Loss: 1.223480
Train Epoch:  1 [  800/4654 (17.2%)]	Loss: 1.590812
Train Epoch:  1 [ 1200/4654 (25.8%)]	Loss: 2.259790
Train Epoch:  1 [ 1600/4654 (34.4%)]	Loss: 1.462389
Train Epoch:  1 [ 2000/4654 (43.0%)]	Loss: 1.569675
Train Ep

KeyboardInterrupt: 

In [37]:
torch.save(model.state_dict(),"resnet50.pt")

### Make Predictions

In [38]:
model = models.resnet50(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
    
n = len(np.unique(train_set.targets))
classifier = nn.Sequential(nn.Linear(model.fc.in_features,128),
                          nn.ReLU(),
                          nn.Dropout(0.2),
                          nn.Linear(128,64),
                          nn.ReLU(),
                          nn.Dropout(0.2),
                          nn.Linear(64,n))
model.fc = classifier
model.to(device)

model.load_state_dict(torch.load('resnet50.pt'))

In [39]:
i2c = {}
for k,v in train_set.class_to_idx.items():
    i2c[v]=[int(k[0]),int(k)]

In [40]:
from PIL import Image
import os
import pandas as pd

In [41]:
submission = pd.read_csv('data/test.csv')
submission['class_name'] = 1
submission['breed'] = 11
submission.head()

n = len(os.listdir('data/test'))
with torch.no_grad():
    for _,i in enumerate(os.listdir('data/test')):
        img = Image.open('data/test/'+i)
        if img.mode != 'RGB':
            continue
        img = test_transform(img)
        img = img.view(1,3,224,224).to(device)
        output = model(img)
        pred = output.argmax(dim=1)
        submission.loc[submission.id==i[:-4],['class_name','breed']] = i2c[pred.item()]
        if _%100==0:
            print(round(100*_/n,1),'%')

0.0 %
9.1 %
18.2 %
27.4 %
36.5 %
45.6 %
54.7 %
63.9 %
73.0 %
82.1 %
91.2 %


In [42]:
submission

,id,class_name,breed
0,img_0966,2,24
1,img_6542,2,23
2,img_2360,2,22
3,img_0763,1,11
4,img_6579,1,11
5,img_1013,2,24
6,img_4896,1,11
7,img_0418,1,11
8,img_1070,2,23
9,img_6988,1,11


In [44]:
submission.to_csv('submission.csv',index=False)